In [1]:
#Non-custom imports.
#
#Imports for unused modules are commented out to minimize errors and setup.

import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np
import math
#from matplotlib.animation import FuncAnimation
import time
#import matplotlib.animation as animation
#import seaborn as sns
#import pandas as pd
#from graphics import *
#import sys
#import random
#import re
from tqdm import tqdm
#import pandas as pd

In [ ]:
A = np.array(
        [[np.cos(θ), 0, 0, 0, 0, 0, 1, 0, 0],
         [np.sin(θ), 0, 0, 0, 0, 0, 0, 0, 1/2],
         [-np.cos(θ), np.cos(θ), 0, 0, 0, 0, 0, 1, 0],
         [-np.sin(θ), -np.sin(θ), 0, 0, 0, 0, 0, 0, 0],
         [0, -np.cos(θ), np.cos(ϕ), 0, 0, 0, 0, 0, 0],
         [0, np.sin(θ), np.sin(ϕ), 0, 0, 0, 0, 0, 0],
         [0, 0, -np.cos(ϕ), np.cos(ϕ), 0, 0, 0, 0, 0],
         [0, 0, -np.sin(ϕ), -np.sin(ϕ), 0, 0, 0, 0, 0],
         [0, 0, 0, -np.cos(ϕ), np.cos(θ), 0, 0, 0, 0],
         [0, 0, 0, np.sin(ϕ), np.sin(θ), 0, 0, 0, 0],
         [0, 0, 0, 0, -np.cos(θ), np.cos(θ), 0, -1, 0],
         [0, 0, 0, 0, -np.sin(θ), -np.sin(θ), 0, 0, 0],
         [0, 0, 0, 0, 0, -np.cos(θ), -1, 0, 0],
         [0, 0, 0, 0, 0, np.sin(θ), 0, 0, 1/2]]
    )

In [185]:
def Tcalc(θ, ϕ, P):
    
    A = np.array([
        [0, np.cos(θ), 0, 1, 1],
        [0, -np.sin(θ), 0, 0, 0],
        [-np.cos(θ), 0, np.cos(ϕ), 0, 1],
        [-np.sin(θ), 0, np.sin(ϕ), 0, 0],
        [0, -np.cos(θ), np.cos(ϕ), 0, 0],
        [0, np.sin(θ), -np.sin(ϕ), 0, 0],
        [2*np.sin(θ), 2*np.sin(θ), 2*np.sin(ϕ), 0, 0]
    ])
    
    B = np.zeros(7)
    B[1] = -P/2
    B[5] = P/2
    B[6] = -P
    
    
    c = np.linalg.lstsq(A, B, rcond=None)[0]
    
    return A, c

In [225]:
np.around(Tcalc(np.deg2rad(33.7), np.deg2rad(33.7), 12.5)[1], 3)

array([-7.529,  1.303, -4.209,  1.678, -2.762])

In [226]:
A, B = Tcalc(np.deg2rad(33.7), np.deg2rad(33.7), 12.5)
A = np.around(A, 3)
B = np.around(B, 3)

np.dot(A, B)

array([ 9.600000e-05, -7.231650e-01,  2.400000e-04,  1.842600e+00,
       -4.585984e+00,  3.059160e+00, -1.158285e+01])

In [256]:
def TcalcJoints(θ, P):
    
    A = np.array([
        [np.cos(θ),  0,          0,            1, 0], #A_x
        [np.sin(θ),  0,          0,            0, 0], #A_y
        [-np.cos(θ), np.cos(θ),  0,            0, 1], #B_x
        [0,          -np.cos(θ), np.cos(θ),    0, 0], #C_x
        [0,          0,          -2*np.sin(θ), 0, 0]  #D_y
    ]) #All excluded rows are linear combinations of other rows
    
    B = np.zeros(5)
    B[1] = -P/2
    B[4] = P
    
    C = np.linalg.solve(A, B)
    
    return A, B, C

In [257]:
np.around(TcalcJoints(np.deg2rad(33.7), 12.5)[2], 3)

array([-11.264, -11.264, -11.264,   9.371,  -0.   ])

In [258]:
A, B, C = TcalcJoints(np.deg2rad(33.7), 12.5)

#A = np.around(A, 5)
#B = np.around(B, 5)
#C = np.around(C, 5)

print(np.around(np.dot(A, C), 3))
print(B)

[ 0.   -6.25 -0.   -0.   12.5 ]
[ 0.   -6.25  0.    0.   12.5 ]


In [7]:
import numpy as np

def TcalcJoints2(θ, P):
    
    A = np.array([
        #AB          BC          CD            AC  CE  BD
        [np.cos(θ),  0,          0,            1,  0,  0], # =0      0 A_x
        [np.sin(θ),  0,          0,            0,  0,  0], # =-P/2   1 A_y
        [-np.cos(θ), np.cos(θ),  0,            0,  0,  1], # =0      2 B_x
        [-np.sin(θ), -np.sin(θ), 0,            0,  0,  0], # =0      3 B_y
        [0,          -np.cos(θ), np.cos(θ),    -1, 1,  0], # =0      4 C_x
        [0,          0,          -2*np.sin(θ), 0,  0,  0], # =P      5 D_y
    ]) #All excluded rows are linear combinations of other rows
    #C_y = -1*B_y + A_y + -(1/2)*D_y
    #D_x is all zeros
    #E_x = -1*C_x
    #E_y = C_y
    #F_x = -1*B_x
    #F_y = B_y
    #G_x = -1*A_x
    #G_y = A_y
    
    B = np.zeros(6)
    B[1] = -P/2
    B[5] = P
    
    C = np.linalg.solve(A, B)
    
    return A, B, C

A, B, C = TcalcJoints2(np.deg2rad(33.7), 12.5)

print(np.around(C, 3))

print()

print(np.around(np.dot(A, C), 3))
print(B)

[-11.264  11.264 -11.264   9.371  28.114 -18.743]

[ 0.   -6.25  0.    0.    0.   12.5 ]
[ 0.   -6.25  0.    0.    0.   12.5 ]
